[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sparks-baird/self-driving-lab-demo/blob/main/notebooks/4.0-paho-mqtt-colab-sdl-demo.ipynb)

# Internet of Things-style communication between Pico W and some other device

The other device could be your laptop, a Google Colab script, etc.

In [1]:
#@markdown This cell checks if you're running in Google Colab or not and assigns it to `IN_COLAB`.

from os import path
try:
  import google.colab
  IN_COLAB = True
  base_dir = "/content/drive/MyDrive/"
except:
  IN_COLAB = False
  base_dir = path.join("data", "external")
  

In [2]:
if IN_COLAB:
  %pip install paho-mqtt

## Pico ID

Load a unique identifier for your Pico device from a `pico_id_secret.py` file that you store in your Google Drive (see below) or current directory (`.`), or specify it manually by setting the `PICO_ID` variable in the form field. If you don't know what the ID is, you can get it by running the following in the Pico REPL (i.e. MicroPython shell, e.g. in Thonny):
```python
from machine import unique_id
from ubinascii import hexlify
my_id = hexlify(unique_id()).decode()
print(my_id)
```

For your convenience, this is done already in [`main.py`](https://github.com/sparks-baird/self-driving-lab-demo/blob/06ededd312087a815b5c0ac8098e82db9c5730c8/src/public_mqtt_sdl_demo/main.py) so you need only run this file once the Pico W has been properly set up and it will be displayed in the terminal.

The file named `pico_id_secret.py` placed directly in your `MyDrive` folder or in your current directory would look like the following:
```python
PICO_ID = "a123b1234azed"
```

In [5]:
#@markdown No need to enter this manually if you've already saved the appropriate file to your Google Drive. The hidden code in this cell will take care of it.
import sys
secret_name = "PICO_ID"
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  sys.path.append('/content/drive/MyDrive/')
  try:
    from pico_id_secret import PICO_ID
  except Exception as e:
    print(e)
    PICO_ID = "" #@param {type:"string"}
    if PICO_ID == "":
      print(f"Couldn't load {secret_name}, and user-input {secret_name} is also empty.")
    else:
      print(f"defaulting to user-input PICO_ID: {PICO_ID}")
else:
  from public_mqtt_sdl_demo.secrets import PICO_ID

In [6]:
"""https://www.steves-internet-guide.com/receiving-messages-mqtt-python-clientq=Queue()"""
import json
import paho.mqtt.client as mqtt
from queue import Queue
from time import time

sensor_data_queue = Queue()
timeout = 30

def on_message(client, userdata, msg):
    sensor_data_queue.put(json.loads(msg.payload))


def observe_sensor_data(R, G, B, pico_id=PICO_ID, hostname="test.mosquitto.org"):

    prefix = f"sdl-demo/picow/{pico_id}/"
    neopixel_topic = prefix + "GPIO/28"
    sensor_topic = prefix + "as7341/"

    # The callback for when the client receives a CONNACK response from the server.
    def on_connect(client, userdata, flags, rc):
        if rc != 0:
          print("Connected with result code " + str(rc))
        # Subscribing in on_connect() means that if we lose the connection and
        # reconnect then subscriptions will be renewed.
        client.subscribe(sensor_topic)

    client = mqtt.Client()  # create new instance
    client.on_connect = on_connect
    client.on_message = on_message
    client.connect(hostname)  # connect to broker
    client.subscribe(sensor_topic)

    # ensures double quotes for JSON compatiblity
    payload = json.dumps(dict(R=R, G=G, B=B))
    client.publish(neopixel_topic, payload)

    t = time()
    while sensor_data_queue.empty():
        client.loop()
        if t - time() > 30:
          raise ValueError("Failed to retrieve message within timeout period")
    return sensor_data_queue.get()


print(observe_sensor_data(5, 10, 15, pico_id=PICO_ID))
print(observe_sensor_data(20, 30, 25, pico_id=PICO_ID))
print(observe_sensor_data(0, 0, 0, pico_id=PICO_ID))


{'ch583': 187, 'ch670': 134, 'ch510': 655, 'ch410': 77, 'ch620': 302, 'ch470': 3521, 'ch550': 439, 'ch440': 1577}
{'ch583': 909, 'ch670': 292, 'ch510': 3015, 'ch410': 190, 'ch620': 1838, 'ch470': 5575, 'ch550': 743, 'ch440': 3176}
{'ch583': 59, 'ch670': 66, 'ch510': 232, 'ch410': 30, 'ch620': 39, 'ch470': 2635, 'ch550': 368, 'ch440': 391}
